In [1]:
import os.path as osp
import dotenv
from snorkel import SnorkelSession
dotenv.load_dotenv('../env.sh')
session = SnorkelSession()

In [2]:
from snorkel.models import candidate_subclass

InducingCytokine = candidate_subclass('InducingCytokine', ['cytokine', 'cell_type'])
train_cands = session.query(InducingCytokine).filter(InducingCytokine.split == 0).all()
dev_cands = session.query(InducingCytokine).filter(InducingCytokine.split == 1).all()

In [4]:
train_cands[:5]

[InducingCytokine(Span("b'IFN-\xce\xb2'", sentence=1201, chars=[6,10], words=[1,1]), Span("b'Th2'", sentence=1201, chars=[100,102], words=[16,16])),
 InducingCytokine(Span("b'IL-4'", sentence=1173, chars=[238,241], words=[47,47]), Span("b'Th2'", sentence=1173, chars=[210,212], words=[43,43])),
 InducingCytokine(Span("b'IL-5'", sentence=1173, chars=[250,253], words=[49,49]), Span("b'Th2'", sentence=1173, chars=[210,212], words=[43,43])),
 InducingCytokine(Span("b'IL-13'", sentence=1173, chars=[262,266], words=[51,51]), Span("b'Th2'", sentence=1173, chars=[210,212], words=[43,43])),
 InducingCytokine(Span("b'IL-17'", sentence=836, chars=[124,128], words=[22,22]), Span("b'Th17'", sentence=836, chars=[143,146], words=[26,26]))]